## 1. Loading the data

### 1.1. Loading libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import glob, os, json
from pathlib import Path
import time

In [9]:
start_time = time.time()

places = []

for file in Path("C:\\Users\\Przem\\Code\\Analysis-And-Visualization-For-Bike-Sharing-Systems-In-Warsaw\\data").glob("*.json"):   # ścieżka dostępu do katalogu, gdzie przechowywane są pliki JSON

    with open(file) as json_file:
        data = json.load(json_file)

    for result in data:
        for place in result["places"]:
            place["date"] = file.stem 
            places.append(place)

df = pd.DataFrame(places)

print("--- %s seconds ---" % (time.time() - start_time))

--- 158.76451110839844 seconds ---


In [10]:
print(df.shape)
print(df.columns)

(1478785, 16)
Index(['uid', 'lat', 'lng', 'name', 'spot', 'number', 'bikes', 'bike_racks',
       'free_racks', 'terminal_type', 'bike_numbers', 'bike_types',
       'place_type', 'rack_locks', 'date', 'maintenance'],
      dtype='object')


In [11]:
df_ = df.drop(df.columns[[4, 5, 9, 11, 12, 13, 15]], axis = 1)

In [12]:
def data_preprocessing_1(df): # wstępne przetworzenie danych
    
# rozbicie nazw JSONów na rok, miesiąc, dzień, godzinę i minuty
    df_ = pd.DataFrame({'year': df['date'].str[0:4],
                     'month': df['date'].str[4:6],
                     'day': df['date'].str[6:8],
                     'hour': df['date'].str[9:11],
                     'minutes': df['date'].str[11:13]})
    df['date'] = pd.to_datetime(df_)  # stworzenie z powyższych "części" nazwy JSONów kolumny z datą
    df['day_of_week'] = df['date'].dt.day_name() # stworzenie nowej kolumny z dniem tygodnia
    df['city_code'] = '352200375' # kod stacji - w tym przypadku dla Warszawy
    df['date_normalize'] = df['date'] 
    # stworzenie kolumny z datą w postaci Y-M-D
    df['date_normalize'] = pd.to_datetime(df['date_normalize']).dt.strftime('%Y-%m-%d') 
    
    return df

In [13]:
def data_preprocessing_2(df):
    df = df[df.date_normalize != '2018-04-02'] # wyrzucenie danych z 2018-04-02 (ponieważ w tym dniu dostępne są dane tylko do godziny 3:00)
    df['bikes'] = df['bikes'].replace('?', np.nan).astype(float)  # wyrzucenie '?' z kolumny 'bikes'
    df['bike_racks'] = pd.to_numeric(df['bike_racks'])  # przekonwertowanie kolumny z ilością stanowisk na stacji na wartości numeryczne
    return df
    

In [14]:
data = data_preprocessing_1(df_)
df = data_preprocessing_2(data)

C:\Users\Przem\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Przem\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
df_temp_march = pd.read_csv('B00300S_2018_03.csv') # wartości temperatur odnotowane dla marca 2018
df_temp_april = pd.read_csv('B00300S_2018_04.csv') # wartości temperatur odnotowane dla kwietnia 2018

In [16]:
df_temp_march.head(3)

,249190890;B00300S;2018-03-01 00:00;-19,10;
0,249190890;B00300S;2018-03-01 00:10;-19,40;
1,249190890;B00300S;2018-03-01 00:20;-19,20;
2,249190890;B00300S;2018-03-01 00:30;-19,20;


In [17]:
df_temp_march = df_temp_march.fillna(0) 
df_temp_april = df_temp_april.fillna(0)

temperature_march = df_temp_march.rename(columns={"249190890;B00300S;2018-03-01 00:00;-19": "temp1", "10;": "temp2"})
temperature_april = df_temp_april.rename(columns={"249190890;B00300S;2018-04-01 00:00;7": "temp1", "20;": "temp2"})

# rozdzielenie pierwszej kolumny na kod stacji i datę odczytu temperatury
temperature_df = pd.DataFrame(columns = ['city_code', 'date'])
temperature_df['date'] = temperature_march['temp1'].str[18:34]
temperature_df['city_code'] = temperature_march['temp1'].str[0:9]

In [18]:
total_temp = pd.DataFrame(columns = ['city_code', 'date', 'temperature']) 
total_temp['date'] = temperature_df['date'].append(temperature_april['temp1'].str[18:34], ignore_index = True)
total_temp['city_code'] = temperature_df['city_code'].append(temperature_april['temp1'].str[0:9], ignore_index = True)

In [19]:
# "ekstrakcja" z ramki danych roku, miesiaca, dnia, godziny i minuty
cops = pd.DataFrame({'year': total_temp['date'].str[0:4],
                 'month': total_temp['date'].str[5:7],
                 'day': total_temp['date'].str[8:11],
                 'hour': total_temp['date'].str[11:13],
                 'minutes': total_temp['date'].str[14:17]})

In [20]:
total_temp['date'] = pd.to_datetime(cops) # stworzenie kolummy z datą odczytu wartości temperatury

In [21]:
# przekonwertowanie drugiej kolumny na str i "wydobycie" z niej części dziesiętnej
temperature_march['temp2'] = temperature_march['temp2'].astype(str).str[0]  
temperature_april['temp2'] = temperature_april['temp2'].astype(str).str[0]

# "wydobycie" z pierwszej kolumny wartości jedności, bądź dziesiątek temperatur
temperature_march['temp1'] = temperature_march['temp1'].str[35:]
temperature_march['temp1'] = temperature_march['temp1'].replace(';','', regex=True)

temperature_april['temp1'] = temperature_april['temp1'].str[35:]
temperature_april['temp1'] = temperature_april['temp1'].replace(';','', regex=True)

In [22]:
# połączenie częsci dziesiątek/jedności z pierwszej kolumny z częścią dziesiętną z drugiej kolumny
new_temp1 = temperature_march['temp2'].copy()
temperature_march['temp1'] = temperature_march['temp1'].str.cat(new_temp1, sep='.')
temperature_march= temperature_march.drop(temperature_march.columns[[1]], axis=1)

new_temp2 = temperature_april['temp2'].copy()
temperature_april['temp1'] = temperature_april['temp1'].str.cat(new_temp2, sep='.')
temperature_april= temperature_april.drop(temperature_april.columns[[1]], axis=1)

In [23]:
temp_all = pd.concat([temperature_march, temperature_april])
temp_all = temp_all.reset_index(drop=True) 


In [24]:
total_temp['temperature'] = temp_all

In [25]:
df_ = pd.merge(df, total_temp, how='left', on=['date', 'city_code']) # dołączanie do głównej ramki danych informacji o temperaturze

In [26]:
df_['temperature'] = pd.to_numeric(df_['temperature']) # zmiana typu danych kolumny z wartościami temp. z str na int